In [5]:
           .ORIG x3000
       
;;----------------------------------------------------------------------------------------------------------------------
;;------------------------------------------- THE MAIN GAME ------------------------------------------------------------
;;----------------------------------------------------------------------------------------------------------------------

            BRnzp start

playerStartAGAIN:  JSR POP                  ;;only come here if the user is
                                            ;;starting a new game within
                                            ;;the current execution
                                            ;;(need to pop from previous game)
start:  JSR PUSH                          
        LEA R0, header
        PUTS
        LEA R0, welcome        
        PUTS
        LEA R0, header
        PUTS
        LEA R0, directions
        PUTS        
        LD R1, lives         
        STI R1 cpLives1            ;;set up the lives
        STI R1, userLives1  
        LD R1, ships              ;;set up the number of ships
        STI R1, shipNum2
        AND R1, R1, #0            ;;set up the values of the ships
        ADD R1, R1, #2
        STI R1, destroyer2
        ADD R1, R1, #1
        STI R1, submarine2
        STI R1, cruiser2
        ADD R1, R1, #1
        STI R1, battleship2
        ADD R1, R1, #1
        STI R1, carrier2
        LD R1, shipPointerSTART    ;;set up the ship pointer
        STI R1, shipPointer2   
        AND R1, R1, #0
        LDI R1, threeFlag1         ;;zero the flag
        LD R1, POINTER5
        ST R1, POINTER7
        JSR zero                   ;;clear the ships from the cp's board
        LD R1, POINTER6
        ST R1, POINTER7
        JSR zero                   ;;clear the cp's past guesses
        JSR getRandom              ;;set up the cp's board
        JSR POP
 
;;---------------------------------------- ;;THE GUESSING
            JSR PUSH
USERguess:  JSR PUSH         
            JSR promptGuess        
            JSR POP
            LDI R1, cpLives1        
            BRp CPguess            ;;the cp still has lives left...so let the cp guess
            BRz gameIsOver         ;;if not, game is over
        
        
CPguess:   JSR PUSH
           JSR outGuess           
           JSR POP
           LDI R1, userLives1
           BRp USERguess          ;;the user still has lives left...so let the user guess
           JSR POP
           
;;----------------------------------------------;; NEW GAME?
           
        
gameIsOver:  JSR PUSH               ;;if not, prompt new game
        JSR newGame        
        LDI R5, startFlag1     ;;new game or stop?
        BRp playerStartAGAIN
        JSR POP
        TRAP x25

;;-----------------------------------------------





lives: .FILL x0011               ;;17
ships: .FILL #5
Carrier2: .FILL Carrier      ;;number of holes per ship
Battleship2: .FILL Battleship
Cruiser2: .FILL Cruiser
Submarine2: .FILL Submarine
Destroyer2: .FILL Destroyer
userLives1: .FILL userLives
cpLives1: .FILL cpLives
shipNum2: .FILL shipNum
threeFlag1: .FILL threeFlag
shipPointer2: .FILL shipPointer
shipPointerSTART: .FILL Carrier
startFlag1: .FILL startFlag
header: .STRINGZ "\n======================\n"
welcome: .STRINGZ "\nWelcome to Battleship!\n"
directions: .STRINGZ "\nTry to find all of my 5 ships hidden in board spaces A-J, 1-9 before I find yours.\n"
POINTER5: .FILL POINTER
POINTER6: .FILL POINTERa
      
;;----------------------------------------------------------------------------------------------------------------------
;;------------------------------------------- THE SUBROUTINES ----------------------------------------------------------
;;----------------------------------------------------------------------------------------------------------------------

;;------------------------------------------ zero the board
zero:     STI R7, REG7_1
          LD R1, POINTER7    ;;set up the pointer
          LDI R2, boardLength5     ;;add the board length to the pointer
                                   ;;to determine the last mem location
                                   ;;of the cp's board
           ADD R4, R1, R2
           ADD R4, R4, #1         ;;off by 1
           STI R4, CPendOfBoard2
          ADD R1, R1, #1            
          AND R5, R5, #0     
zerolp:   LDI R2, CPendOfBoard2
          NOT R2, R2
          ADD R2, R2, #1
          ADD R3, R1, R2      ;;at the end of the board?
          BRz doneZERO
          STR R5, R1, #0      ;;put a zero at that location
          ADD R1, R1, #1      ;;increment the pointer
          BRnzp zerolp
doneZERO: LDI R7, REG7_1
          RET        



POINTER7: .FILL #0
CPendOfBoard2: .FILL CPendOfBoard
boardLength5: .FILL boardLength

;;---------------------------------------------------------------------------------  ;;GET CP's SHIP LOCATIONS      
getRandom:      STI R7, REG7_1               ;;*****note: don't put push or pop inside this function because
                                             ;;the program does not reach this portion of code
                                             ;;proportionately to the number of times
                                             ;;this function is run
                                             ;;so pushing and popping will result
                                             ;;in misuse of the call stack*************
getRandomLoop:  LDI R0, shipNum1
            BRz doneRandom
            ;random number is in R1           
            JSR Random            
            ;put the random num within the board range
            LDI R1, X1
            LDI R0, BitMask1
            AND R1, R0, R1              
            STI R1, RandomSpace1           ;;where the ship will start   
            JSR Random   
            ;find the orientation of the ship
            LDI R1, X1
            AND R2, R2 #0
            ADD R2, R2 #1
            AND R1, R2, R1              
            STI R1, orientation1           ;;the orientation of the ship (0 = horizontal, 1 = vertical)
            JSR cpBoard                     
            LDI R0, shipNum1
            ADD R0, R0, #-1
            STI R0, shipNum1            
            BRnzp getRandomLoop
doneRandom: LDI R7, REG7_1
            RET   

BitMask1: .FILL BitMask
X1: .FILL X
orientation1: .FILL orientation

;;------------------------------------------- ;;INPUT CP's SHIPS
cpBoard:              LD R1, POINTER1
                      LDI R2, boardLength4     ;;add the board length to the pointer
                                               ;;to determine the last mem location
                                               ;;of the cp's board
                      ADD R4, R1, R2
                      STI R4, CPendOfBoard1
                      LDI R2, RandomSpace1
                      BRp checkLocation
                      ADD R2, R2, #1             ;;add 1 if negative
checkLocation:        ADD R1, R2, R1             ;add the RandomSpace to the pointer
                      ST R1, RandomPointer
                      LDR R3, R1, #0             ;;check for collision
                      BRz checkOrientation       ;;if no collison, now deal with orientation
                      BRnzp collision            ;;if there is a collision,
                                                 ;;get a new board space and orientation
                       BRnzp cpBoard
checkOrientation:      LDI R4, orientation1
                       BRz horizontal
                       BRp vertical
horizontal:            LDI R5, shipPointer       ;;set up the number to be stored in each space   
                       LDI R4, shipPointer       ;;set up counter
                       ADD R4, R4, #1            ;;add 1 so we can decrement at the beginning of the
                                                 ;;loop instead of the end
                       LD R1, RandomPointer      ;;where we are starting from (pointer + random val)
                         
                       ;;check that the entire ship can fit w/ no collisions with other ships
horizlpCHECK:          ADD R4, R4, #-1
                       BRz horizlpSETUP             
                       LDR R3, R1, #0             ;;collides with another ship?
                       BRp collision
                       LDI R2, CPendOfBoard1      ;;collides with the end of the board?  
                       NOT R2, R2
                       ADD R2, R2, #1
                       ADD R3, R2, R1
                       BRz collision  
                       ADD R1, R1, #1                         
                       BRnzp horizlpCHECK
                         
horizlpSETUP:          LDI R4, shipPointer      ;;restore the counter
                       ADD R4, R4, #1
                       LD R1, RandomPointer    ;;restore the starting point
horizlp:               ADD R4, R4, #-1         ;;there were no collisions, so actually input the ship
                       BRz doneInput                                             
                       STR R5, R1, #0           ;;store the size of the ship in each board space the ship is on
                       ADD R1, R1, #1           ;;increment to the next horizontal board space                         
                       BRnzp horizlp
                       ;;SAME AS HORIZONTAL, EXCEPT ADD 9 TO THE POINTER INSTEAD OF 1
vertical:              LDI R5, shipPointer
                       LDI R4, shipPointer
                       ADD R4, R4, #1
                       LD R1, RandomPointer
vertlpCHECK:           ADD R4, R4, #-1
                       BRz verticallpSETUP
                       LDR R3, R1, #0
                       BRp collision
                       LDI R2, CPendOfBoard1
                       NOT R2, R2
                       ADD R2, R2, #1
                       ADD R3, R2, R1
                       BRz collision
                       ADD R1, R1, #9
                       BRnzp vertlpCHECK
            
verticallpSETUP:       LDI R4, shipPointer 
                       ADD R4, R4, #1
                       LD R1, RandomPointer 
verticallp:            ADD R4, R4, #-1
                       BRz doneInput                       
                       STR R5, R1, #0
                       ADD R1, R1, #9      ;;increment by 9 to get to next vertical board space            
                       BRnzp verticallp              
collision:             JSR getRandomLoop    ;;get a new board space and orientation
doneInput:             LD R4, shipPointer   ;;increment the shipPointer (for the next ship)
                       ADD R4, R4, #1
                       ST R4, shipPointer
                       RET           
         
POINTER1 : .FILL POINTER
CPendOfBoard1: .FILL CPendOfBoard
boardLength4: .FILL boardLength
RandomPointer: .FILL #0
RandomSpace1: .FILL RandomSpace
shipCounter: .FILL #0
shipPointer: .FILL Carrier       ;;the ship currently being input, starting with carrier (see ship array)
Carrier1: .FILL Carrier
Battleship1: .FILL Battleship
Cruiser1: .FILL Cruiser
Submarine1: .FILL Submarine
Destroyer1: .FILL Destroyer
shipVal: .FILL #0 ;;the number of holes of the ship currently being input

;;----------------------------------------------------- ;;GET GUESS FROM USER

;;get the 2 chars from the user
;;multiply the first char by 32 (shift left twice)
;;add the second char to the first
;;full string is now in mem location guessSpace

promptGuess: STI R7, REG7_1             
             AND R2, R2, #0
             ADD R2, R2, #1
             ST R2, promptCounter ;;save the counter
             LD R0, prompt1
             PUTS             
;;get chars from user  
error:       LD R2, promptCounter                ;;return here if the user input an invalid board space
iterate:     GETC  ;;put the char in R0          ;;the loop to get the user's chars
             ADD R2, R2, #-1             
             BRn doneMult             
             JSR PUSH
             ;;multiply R0 by 32 (shift left twice to accommodate second space)
             LD R1, convert
             JSR Multiply
             STI R2, guessSpace1  ;;save the multiplied char in guessSpace
             JSR POP
             BRnzp iterate
doneMult:    LDI R3, guessSpace1
             ADD R3, R3, R0                  ;;add the second char to the first
             STI R3, guessSpace1
             GETC                          ;;get rid of the final zero (always input by GETC)         
             JSR checkChar                 ;;check that the input is valid     
                                           ;;***note - similar, don't push/pop here. not called proportionately
                                           ;;(if the user input is invalid)
             LDI R7, REG7_1                ;;only comes here if a valid board space was input
             RET
             
shipNum1: .FILL shipNum
quit1: .FILL quit
new1: .FILL new
prompt1: .FILL prompt
promptCounter: .FILL #0
convert: .FILL x0100
REG7_1: .FILL REG7

;;----------------------------------------------------- ;;CHECK IF GUESS IS VALID

;;check through all the values to see if the user's input value matches a board space
;;if it does, then go to the corresponding board space in the cp's board and check
;;if that space has a ship in it (output hit/miss)
;;if not, return back to prompt char and keep prompting user until they input a valid board space
;;R1 = pointer, R2 = value we are trying to find, R4 = counter, R5 = contents at pointer (R1)
;;R6 contains the number of times the pointer is incremented (which is the parameter for the HITmiss function)

checkChar:  ST R7, REG7_2
            LD R1, A1ck1            ;;A1ck contents have the mem location of A1ck, so R1 points to A1ck address
            ST R1, current
            LDI R2, guessSpace1, #0                       ;;what we are comparing
            LDI R4, boardLength1, #0
            ST R4, boardLengthck                     ;;store the board length in boardLengthck
            AND R6, R6, #0
            ADD R6, R6, #1                           ;;start at 1 to increment from POINTER in HIT
checkCharlp: LDR R5, R1, #0                             ;;load the contents at that pointer into R5
            LD R4, boardLengthck                        ;;R4 is the counter
            BRz endOfBoard                            ;;are we at the end of the board?
            ;;2s complement of R5 (ascii board space value)
            NOT R5, R5
            ADD R5, R5, #1
            ADD R3, R5, R2
            BRz foundIT
            ADD R1, R1, #1
            ADD R6, R6, #1
            ADD R4, R4, #-1                           ;;decrement the counter
            ST R4, boardLengthck
            ST R1, current
            BRnzp checkCharlp
endOfBoard: LEA R0, invalid                   ;;the input is not valid...
            PUTS
            AND R0, R0, #0                   ;;*****note: don't put push or pop here because
                                             ;;the program does not reach this portion of code
                                             ;;proportionately to the number of times
                                             ;;this function is run
                                             ;;so pushing and popping will result
                                             ;;in misuse of the call stack*************
            JSR error                        ;;...so prompt the user to guess again            
foundIT:    ;;do something
            LDR R1, R1, #0                   ;;R1 contains the matching contents
            ST R1, foundValue            
            ST R6, foundLocation            
            JSR PUSH
            JSR HITmiss
            JSR POP
            LD R7, REG7_2
            RET   
             

guessSpace1: .FILL guessSpace 
A1ck1: .FILL A1ck
boardLength1: .FILL BoardLength
current: .FILL #0
foundValue: .FILL #0
foundLocation: .FILL #0
invalid: .STRINGZ "ERROR - Invalid board space. Please input a valid board space. \n"
boardLengthck: .FILL #0
REG7_2: .FILL #0


;;----------------------------------------------------------- ;;CHECK THE CP's BOARD FOR THIS GUESS (hit or miss?)

HITmiss: ST R7, REG7_3
         LD R1, POINTER1
         LD R2, foundLocation
         ADD R1, R1, R2                     ;;increment the pointer by the foundLocation (the user's guess)
                                             ;;check if the location contains a 1 or a 0 (hit or miss)
         LDR R3, R1, #0
         ADD R3, R3, #-1
         BRz shipFound                       ;;there was a 1 there
         BRn noShipFound                     ;;there was a 0 there
shipFound:  ST R1, foundLocation             ;;save the location of the hit ship
            AND R0, R0, #0
            .SET MODE TERMINAL
            LD R0, launchGif2
            TERMINAL R0, 0
            LEA R0, dots
            PUTS
            LEA R0, HIT
            PUTS                             ;;abolishes R7
            .SET MODE TERMINAL               ;;hitGIF
            LD R0, hitGif1
            TERMINAL R0, 0
            ;;CHECK WHICH TYPE OF SHIP IT WAS
            LDR R1, R1, #0
            STI R1, hitSHIP1
            JSR checkHIT
            LD R1, foundLocation             ;;the location of the hit ship
            AND R2, R2, #0                   ;;zero the ship (remove the hit ship from the board)
            STR R2, R1, #0
            LDI R5, cpLives3
            ADD R5, R5, #-1
            BRz youWon
            STI R5, cpLives3
            BRnzp doneHITmiss
noShipFound: AND R0, R0, #0
             .SET MODE TERMINAL
             LD R0, launchGif2
             TERMINAL R0, 0
             LEA R0, dots
             PUTS
             LEA R0, MISS
             PUTS
             .SET MODE TERMINAL
             LD R0, missGif2
             TERMINAL R0, 0
doneHITmiss: LD R7, REG7_3
             RET
youWon:      LEA R0, youWin
             PUTS
             STI R5, cpLives3         ;;update cpLives to zero!
             LD R7, REG7_3
             RET

hitGif1: .FILL hitGif
hitSHIP1: .FILL hitSHIP
missGif2: .FILL missGif
launchGif2: .FILL launchGif
cpLives3: .FILL cpLives
dots: .STRINGZ "..."
HIT: .STRINGZ "Hit!\n"
MISS: .STRINGZ "Miss!\n"
youLost: .STRINGZ "Game Over!\n"
youWin: .STRINGZ "\nCongratulations, YOU WON!\n"
REG7_3: .FILL #0
;;--------------------------------------------------------------CHECK HIT
checkHit:    ST R7, REG7_6
             LDI R1, hitSHIP1
             AND R2, R2, #0           ;;check the contents of the hit ship against all of the
                                      ;;ship values (from 2 to 3)
             ADD R2, R2, #5
             NOT R2, R2               ;;2s complement of R2
             ADD R2, R2, #1
             AND R3, R3, #0           ;;the counter (used to increment the shipPointer)
checkHITlp:  ADD R4, R1, R2
             BRz doneCheckHit
             ADD R3, R3, #1
             ST R3, incremented
             ADD R2, R2, #1
             BRnzp checkHITlp
                                         ;;increment the shipPointer by the counter,
                                         ;;and decrement the ship value in that location
                                         ;;if the ship value in that location = 0,
                                         ;;output text to the user indicating that ship has
                                         ;;been sunk
doneCheckHit: ADD R5, R3, #-2            ;;is the hit ship a 3? (counter = 2 means that ship = 3) 
                                         ;;if so, check if a 3 space ship 
                                         ;;has already been sunken
              BRz threeFound
falseAlarm:   LD R5, shipPointer3        
              ADD R5, R5, R3             
              LDR R2, R5, #0             ;;load the contents into R2
              ADD R2, R2, #-1
              BRz SUNK
              STR R2, R5, #0             ;;store the updated contents back into
                                         ;;the location R5 is pointing to
              BRnzp returnCheckHit
threeFound:   LD R5, threeFlag           ;;was the 3 space ship already sunk?
              BRz falseAlarm
              LD R3, incremented
              ADD R3, R3, #2
              ST R3, incremented
              LD R5, shipPointer3
              ADD R5, R5, R3
              LDR R2, R5, #0
              ADD R2, R2, #-1
              BRz SUNK
              STR R2, R5, #0
              BRnzp returnCheckHit
SUNK:         LEA R1, CarrierSUNK
              LD R2, incremented
              ADD R1, R1, R2
              LDR R0, R1, #0
              PUTS
              LD R3, incremented
              ADD R3, R3, #-2
              BRnp returnCheckHit        ;;if not 2
              ADD R3, R3, #1
              ST R3, threeFlag           ;;necessary because two ships have equal lengths
                                         ;;if one of the 3-space ships is sunk,
                                         ;;add a flag indicating that the next time a 3-space
                                         ;;ship is hit, that ship should be the other 3-space ship
                                         ;;not the already sunk on
returnCheckHit: LD R7, REG7_6
                RET


REG7_6: .FILL #0
shipPointer3: .FILL Carrier
incremented: .FILL #0
CarrierSUNK: .FILL CarrierSUNK1            ;;need these because want to treat each label
                                           ;;as 1 mem location, and .STRINGZ uses multiple
                                           ;;mem locations
BattleshipSUNCK: .FILL BattleshipSUNK1
CruiserSUNK: .FILL CruiserSUNK1
DestroyerSUNK: .FILL DestroyerSUNK1
SubmarineSUNK: .FILL SubmarineSUNK1
CarrierSUNK1: .STRINGZ "\n\n--You sunk my carrier!--\n\n"
BattleshipSUNK1: .STRINGZ "\n\n--You sunk my battleship!--\n\n"
CruiserSUNK1: .STRINGZ "\n\n--You sunk my cruiser!--\n\n"
SubmarineSUNK1: .STRINGZ "\n\n--You sunk my submarine!--\n\n"
DestroyerSUNK1: .STRINGZ "\n\n--You sunk my destroyer!--\n\n"
threeFlag: .FILL #0


;;--------------------------------------------------------NEW GAME             
newGame: STI R7, REG7_0
newGamelp: LEA R0, startAgainSTRING
         PUTS
         GETC         
         LD R1, NASCII
         NOT R1, R1
         ADD R1, R1, #1
         ADD R1, R1, R0
         BRz startAgain
         LD R1, SASCII
         NOT R1, R1
         ADD R1, R1, #1
         ADD R1, R1, R0
         BRz endGame
         LEA R0, InvalidChar
         PUTS       
         BRnzp newGamelp
startAgain: AND R5, R5, #0
            ADD R5, R5, #1
            ST R5, startFlag      ;;add a flag indicating start again
            LDI R7, REG7_0
            RET
endGame:    AND R5, R5, #0
            ST R5, startFlag      ;;clear the flag
            LDI R7, REG7_0
            RET

REG7_0: .FILL REG7
startAgainSTRING: .STRINGZ "Type 'N' for a new game or 'S' to stop.\n"
NASCII: .FILL x004E
SASCII: .FILL x0053
InvalidChar: .STRINGZ "ERROR - Invalid character. Please input a valid character.\n"
startFlag: .FILL #0

;;; ----------------------------------------- THE STACK!-------------------------------------------------------
;; The Stack for the whole system:

PUSH:  ST R6, REG6     ;; save R5, R6 to MEMORY
       ST R5, REG5     ;; R7 has return address
                       ;; REG7 is in MEMORY already
                       ;; R6 and R5 are destroyed
       LD R6, TOP     ;; R6 is now TOP of stack
       STR R0, R6, #0      ;; push R0 on stack
       ADD R6, R6, #1  ;; incr
       STR R1, R6, #0      ;; push R1 on stack
       ADD R6, R6, #1  ;; incr
       STR R2, R6, #0      ;; push R2 on stack
       ADD R6, R6, #1  ;; incr
       STR R3, R6, #0      ;; push R3 on stack
       ADD R6, R6, #1  ;; incr
       STR R4, R6, #0      ;; push R4 on stack
       ADD R6, R6, #1  ;; incr
       STR R5, R6, #0      ;; push R5 on stack
       ADD R6, R6, #1  ;; incr
       LD R5, REG6     ;; get R6 from MEMORY (destroys R5)
       STR R5, R6, #0      ;; push R6 on stack
       ADD R6, R6, #1  ;; incr
       LD R5, REG_7     ;; get R7 from MEMORY
       STR R5, R6, #0  ;; push R7 on stack
       ADD R6, R6, #1  ;; incr
       ST R6, TOP      ;; save incremented TOP
       LD R6, REG6
       LD R5, REG5     ;; 
       RET
POP:   LD R6, TOP     ;; R6 is TOP of stack
       ADD R6, R6, #-1     ;; decr
       LDR  R1, R6, #0      ;; load R7
       ST  R1, REG_7    ;; save to MEMORY
       ADD R6, R6, #-1     ;; decr
       LDR  R1, R6, #0      ;; load R6
       ST  R1, REG6    ;; save to MEMORY
       ADD R6, R6, #-1     ;; decr
       LDR  R5, R6, #0      ;; load R5
       ADD R6, R6, #-1     ;; decr
       LDR  R4, R6, #0      ;; load R4
       ADD R6, R6, #-1     ;; decr
       LDR  R3, R6, #0      ;; load R3
       ADD R6, R6, #-1     ;; decr
       LDR  R2, R6, #0      ;; load R2
       ADD R6, R6, #-1     ;; decr
       LDR  R1, R6, #0      ;; load R1
       ADD R6, R6, #-1     ;; decr
       LDR  R0, R6, #0      ;; load R0
       ST R6, TOP      ;; save decremented TOP
       LD R6, REG6, #0     ;; put R6 back the way it was      
       RET
       
TOP:    .FILL    STACK
STACK:  .BLKW    #100
REG5: .FILL    #0
REG6: .FILL    #0
REG_7: .FILL #0


;;------------------------------------------------------------------------------------------ OUT GUESS (cp's guess)

OUTguess: STI R7, REG7_5
           JSR Random             
           ;put the random num within the board range
           LDI R1, X2
           LDI R0, BitMask2
           AND R1, R0, R1   
           ST R1, randomGuess        
           ;;check that the random number hasn't been guessed already
repeatFOUND: LD R1, POINTERa1
           LD R2, randomGuess
           ADD R1, R1, R2
           LDR R1, R1, #0
           BRp repeatGuess
           ;;put the random number relative to its board space
           ;;put a 1 in the guess location to indicate
           ;;that the cp has already guessed that location
           LD R1, POINTERa1
           LD R2, randomGuess
           ADD R1, R1, R2
           AND R5, R5, #0
           ADD R5, R5, #1
                                                             ;;now actually output the guess
           STR R5, R1, #0
           LD R1, A1ckn1                                      ;;R1 is pointing to A1ck
           LD R2, randomGuess
           ADD R1, R1, R2                                    ;;pointer has been incremented by the random guess
           LDR R1, R1, #0                                    ;;put the contents at that pointer into R1
           ST R1, randomGuess                                ;;now output the contents of where the pointer is (2 ASCII chars)
                                                             ;;to the user    
           
           LEA R0, newLine                               
           PUTS           
           LEA R0, subHeader                            ;;subheader makes the guess stand out
           PUTS
           LEA R0, randomGuess
           PUTSP                                        ;;PUTSP because 2 chars in 1 mem location
           LEA R0, questionMark
           PUTS     
           LEA R0, subHeader
           PUTS
           JSR PUSH
           JSR userHITmiss
           JSR POP
           LDI R7, REG7_5
           RET
repeatGuess: LD R1, randomGuess             ;;a repeat was found, so increment
                                            ;;or decrement through previous guesses until
                                            ;;a new guess is found
             LDI R2, boardLengthcheck       ;;check if at end of board
             NOT R2, R2                     ;;if not at end of board, increment random guess
                                            ;;by 1
             ADD R2, R2, #1
             ADD R3, R1, R2
             BRz edgeBoard
             ADD R1, R1, #1
             ST R1, randomGuess
             BRnzp repeatFOUND
edgeBoard:   LD R1, randomGuess
             ADD R1, R1, #-1                ;;decrement until an unguessed space is found
             ST R1, randomGuess
             LD R1, POINTERa1
             LD R2, randomGuess
             ADD R1, R1, R2
             LDR R1, R1, #0
             BRp edgeBoard
             BRnzp repeatFOUND
             
           
dots1: .FILL dots
launchGif1: .FILL launchGif
X2: .FILL X
BitMask2: .FILL BitMask
foundValue1: .FILL foundValue
boardLengthcheck: .FILL boardLength
asciiGuess: .FILL #0
cpFlag: .FILL #0
subHeader: .STRINGZ "\n===\n"
newLine: .STRINGZ "\nMY TURN!\n"
questionMark: .STRINGZ "?"
randomGuess: .FILL #0
nullTerminator: .FILL #0  ;;this must be after randomGuess (because of PUTSP)
A1ckn1: .FILL A1ckn
POINTERa1: .FILL A1a
REG7_5: .FILL REG7

;;; ----------------------------------------------------- ;;RANDOM SUBROUTINE
;;; Memory X has next random number
Random: ST R7,BACK  ; save return location
    LD R0, M
    LD R1, A
    JSR Divide          ; R0 / R1
    ;; q = m / a
    LD R0, QUOTIENT     ; R0 / R1
    ST R0, Q 
    ;; r = m mod a
    LD R0, REMAINDER    ; R0 mod R1
    ST R0, R
        ;; x / q
    LD R0, X
    LD R1, Q
    JSR Divide          ; R0 / R1
    LD R1, QUOTIENT
    ST R1, TEMP2
    LD R1, REMAINDER    ; x mod q
    ST R1, TEMP1
    ;;      a * TEMP1 - r * TEMP2
    LD R0, A
    JSR Multiply        ; R2 <- R0 * R1
    ST R2, TEMP1
    ;;      a * TEMP1 - r * TEMP2
    LD R0, R
    LD R1, TEMP2
    JSR Multiply        ; R2 <- r * TEMP2
    NOT R2,R2           ; -R2
    ADD R2,R2,#1
    ST R2, TEMP2 
    LD R1, TEMP1
    ADD R2, R2, R1      ; TEMP1 - TEMP2
TEST:  BRzp DONE        ; if x < 0 then
    LD R1, M
    ADD R2, R2, R1      ; 
DONE: ST R2, X
    LD R7, BACK         ; Restore return address    
    RET
A: .FILL #7             ;; a , the multiplicative constant is given
M: .FILL #32767         ;; m = 2 Ë† 15 âˆ’ 1, the modulus is given
X: .FILL #10            ;; x, the seed is given
R: .FILL #0
Q: .FILL #0


;;; ----------------------------------------------------- ;;MULTIPLY
;;; R2 <- R0 * R1
;;; Also uses R3 to store SIGN
Multiply: AND R2,R2,#0
  AND R3,R3,#0
  ADD R0,R0,#0         ; compare R0
  BRn MultNEG1
  BR  MultCont
MultNEG1: NOT R3,R3         ; flip SIGN
  NOT R0,R0
  ADD R0,R0,#1
MultCONT: ADD R1,R1,#0         ; compare R1
  BRn MultNEG2
  BR MultInit
MultNEG2: NOT R3,R3         ; flip SIGN
  NOT R1,R1
  ADD R1,R1,#1
MultInit: ADD R0,R0,#0      ; have R0 set the condition codes
MultLoop: BRz MultDone
  ADD R2,R2,R1
  ADD R0,R0,#-1
  BR MultLoop
MultDone: ADD R0,R3,#0
  BRzp MultRet
  NOT R2,R2
  ADD R2,R2,#1
MultRet:  RET            ; R2 has the sum

;;; ----------------------------------------------------- ;;DIVIDE
;;; R0 / R1
;;; Also uses R3 to store SIGN
;;;           R4 to store -R1
;;;           R5 is QUOTIENT
;;;           R6 is REMAINDER
;;;           R2 temp
Divide:   AND R3,R3,#0
  ST R3, QUOTIENT
  ST R3, REMAINDER
  ADD R0,R0,#0         ; compare R0
  BRn DivNEG1
  BR  DivCont
DivNEG1:  NOT R3,R3         ; flip SIGN
  NOT R0,R0
  ADD R0,R0,#1
DivCONT:  ADD R1,R1,#0         ; compare R1
  BRn DivNEG2
  BR DivInit
DivNEG2:  NOT R3,R3         ; flip SIGN
  NOT R1,R1
  ADD R1,R1,#1
DivInit:  ADD R4,R1,#0
  NOT R4,R4
  ADD R4,R4,#1
DivLoop:  ADD R2,R0,R4      ; have R2 set the condition codes
  BRn DivDone
  ADD R0,R0,R4
  LD R2,QUOTIENT
  ADD R2,R2,#1
  ST R2,QUOTIENT
  BR DivLoop
DivDone:  ADD R3,R3,#0         ; Negative?
  BRzp DivRet
  LD R2,QUOTIENT     ; Yes, then negate R2
  NOT R2,R2
  ADD R2,R2,#1
  ST R2,QUOTIENT
DivRet:  ST R0,REMAINDER
         RET            ; R2 has the sum
         

         

;;------------------------------------------------------- USERHITMISS (input if cp's guess was a hit or a miss)
                
userHITmiss: STI R7, REG7_4
userHITmisslp: AND R0, R0, #0
              .SET MODE TERMINAL
              LD R0, launchGif1
              TERMINAL R0, 0              
              LD R0, dots1
              PUTS
              LEA R0, userPrompt
              PUTS             
              GETC             
              ST R0, hitORmiss                   ;;if the user doesn't want to quit or start new game, check for hit or miss
              LD R1, hitORmiss
             LD R2, Hascii
             NOT R2, R2
             ADD R2, R2, #1
             ADD R1, R2, R1                       ;;add R1 to R2
             BRz thatWasAhit
             LD R1, hitORmiss
             LD R2, Mascii
             NOT R2, R2
             ADD R2, R2, #1
             ADD R1, R2, R1
             BRz thatWasAmiss
             LD R0, userError
             PUTS             
             BRnzp userHITmisslp
thatWasAhit:  AND R0, R0, #0
             .SET MODE TERMINAL           
              LD R0, hitGif3
              TERMINAL R0, 0
              LDI R1, userLives2
              ADD R1, R1, #-1
              BRz Iwon
              STI R1, userLives2
              LDI R7, REG7_4
              RET
thatWasAmiss: AND R0, R0, #0              ;;the assembler doesn't like with a label begins with .SET MODE TERMINAL
              .SET MODE TERMINAL           
              LD R0, missGif3
              TERMINAL R0, 0
              LDI R7, REG7_4
              RET       
Iwon:         LEA cpWon
              PUTS
              STI R1, userLives2            ;;update userLives2 to zero!
              LDI R7, REG7_4
              RET
          

hitGif3: .FILL hitGif
missGif3: .FILL missGif
quit2: .FILL quit
new2: .FILL new
userPrompt: .STRINGZ "Input an 'H' for hit or 'M' for miss.\n"
missGif1: .FILL missGif
userLives2: .FILL userLives
userError: .FILL InvalidChar
cpWon: .STRINGZ "\nHA-HA! I won!\n"
REG7_4: .FILL #0
hitORmiss: .FILL #0
Hascii: .FILL x0048
Mascii: .FILL x004D


;;-------------------------------------------------------------------------------------------------- LABELS

BACK: .FILL #0
cpLives: .FILL #0
userLives:   .FILL #0
BitMask: .FILL x005A
guessSpace: .FILL #0
RandomSpace: .FILL #0 
orientation: .FILL #0
hitSHIP: .FILL #0
REG7:    .FILL #0  
QUOTIENT: .FILL #0
REMAINDER: .FILL #0
BoardLength: .FILL x005A
CPendOfBoard: .FILL #0           ;;the memory location of the end of the cp's board (used for comparison
                                  ;;during ship input)
shipNum: .FILL #5
TEMP1: .FILL #0
TEMP2: .FILL #0
prompt: .STRINGZ "\nYOUR TURN!\nWhere would you like to shoot?\n"
hitGif: .STRINGZ "<img width = '400' src= 'https://i.kinja-img.com/gawker-media/image/upload/t_original/k5krfu4agotprfzkscvd.gif'>"
launchGif: .STRINGZ "<img width = '400' src= 'https://i.imgur.com/r6TcKQx.gif'>"
missGif: .STRINGZ "<img width = '400' src= 'http://www.gifbin.com/bin/022011/1296759235_rocket-launch-fail.gif'>"
new: .FILL x004E
quit: .FILL x0053

;;-------------------------------------------------------------------------------------------- CHECK VALUES

A1ck:    .FILL  x4131
A2ck:    .FILL  x4132
A3ck:    .FILL  x4133
A4ck:    .FILL  x4134
A5ck:    .FILL  x4135
A6ck:    .FILL  x4136
A7ck:    .FILL  x4137
A8ck:    .FILL  x4138
A9ck:    .FILL  x4139    
B1ck:    .FILL  x4231
B2ck:    .FILL  x4232
B3ck:    .FILL  x4233
B4ck:    .FILL  x4234
B5ck:    .FILL  x4235
B6ck:    .FILL  x4236
B7ck:    .FILL  x4237
B8ck:    .FILL  x4238
B9ck:    .FILL  x4239
C1ck:    .FILL  x4331
C2ck: .FILL     x4332
C3ck:  .FILL    x4333
C4ck: .FILL     x4334
C5ck: .FILL     x4335
C6ck:  .FILL    x4336
C7ck:  .FILL    x4337
C8ck:  .FILL    x4338
C9ck:  .FILL    x4339
D1ck:  .FILL   x4431
D2ck:  .FILL   x4432
D3ck:  .FILL   x4433
D4ck:  .FILL    x4434
D5ck:  .FILL   x4435
D6ck:  .FILL   x4436
D7ck:   .FILL  x4437
D8ck:  .FILL   x4438
D9ck:   .FILL  x4439
E1ck:   .FILL   x4531
E2ck:   .FILL   x4532
E3ck:   .FILL   x4533
E4ck:   .FILL   x4534
E5ck:   .FILL   x4535
E6ck:   .FILL   x4536
E7ck:    .FILL  x4537
E8ck:   .FILL   x4538
E9ck:   .FILL   x4539
F1ck:   .FILL    x4631
F2ck:   .FILL    x4632
F3ck:   .FILL    x4633
F4ck:   .FILL    x4634
F5ck:   .FILL    x4635
F6ck:   .FILL    x4636
F7ck:   .FILL    x4637
F8ck:   .FILL   x4638
F9ck:   .FILL    x4639
G1ck:   .FILL     x4731
G2ck:   .FILL     x4732
G3ck:   .FILL     x4733
G4ck:   .FILL    x4734
G5ck:   .FILL    x4735
G6ck:   .FILL    x4736
G7ck:   .FILL    x4737
G8ck:    .FILL   x4738
G9ck:   .FILL     x4739
H1ck:   .FILL     x4831
H2ck:    .FILL    x4832
H3ck:    .FILL    x4833
H4ck:    .FILL    x4834
H5ck:    .FILL    x4835
H6ck:    .FILL    x4836
H7ck:    .FILL    x4837
H8ck:    .FILL    x4838
H9ck:    .FILL   x4839
I1ck:    .FILL   x4931
I2ck:    .FILL   x4932
I3ck:    .FILL   x4933
I4ck:    .FILL  x4934
I5ck:    .FILL  x4935
I6ck:    .FILL  x4936
I7ck:    .FILL   x4937
I8ck:    .FILL   x4938         
I9ck:    .FILL     x4939
J1ck:   .FILL     x4A31
J2ck:   .FILL     x4A32
J3ck:   .FILL     x4A33
J4ck:    .FILL   x4A34
J5ck:   .FILL     x4A35
J6ck:    .FILL    x4A36
J7ck:   .FILL     x4A37
J8ck:   .FILL     x4A38
J9ck:    .FILL    x4A39



;;--------------------------------------------------------------------------------------  CP's BOARD

POINTER: .FILL #0
A1: .FILL #0
A2: .FILL #0
A3: .FILL #0
A4: .FILL #0
A5: .FILL #0
A6: .FILL #0
A7: .FILL #0
A8: .FILL #0
A9: .FILL #0
B1: .FILL #0
B2: .FILL #0
B3: .FILL #0
B4: .FILL #0
B5: .FILL #0
B6: .FILL #0
B7: .FILL #0
B8: .FILL #0
B9: .FILL #0
C1: .FILL #0
C2: .FILL #0
C3: .FILL #0
C4: .FILL #0
C5: .FILL #0
C6: .FILL #0
C7: .FILL #0
C8: .FILL #0
C9: .FILL #0
D1: .FILL #0
D2: .FILL #0
D3: .FILL #0
D4: .FILL #0
D5: .FILL #0
D6: .FILL #0
D7: .FILL #0
D8: .FILL #0
D9: .FILL #0
E1: .FILL #0
E2: .FILL #0
E3: .FILL #0
E4: .FILL #0
E5: .FILL #0
E6: .FILL #0
E7: .FILL #0
E8: .FILL #0
E9: .FILL #0
F1: .FILL #0
F2: .FILL #0
F3: .FILL #0
F4: .FILL #0
F5: .FILL #0
F6: .FILL #0
F7: .FILL #0
F8: .FILL #0
F9: .FILL #0
G1: .FILL #0
G2: .FILL #0
G3: .FILL #0
G4: .FILL #0
G5: .FILL #0
G6: .FILL #0
G7: .FILL #0
G8: .FILL #0
G9: .FILL #0
H1: .FILL #0
H2: .FILL #0
H3: .FILL #0
H4: .FILL #0
H5: .FILL #0
H6: .FILL #0
H7: .FILL #0
H8: .FILL #0
H9: .FILL #0
I1: .FILL #0
I2: .FILL #0
I3: .FILL #0
I4: .FILL #0
I5: .FILL #0
I6: .FILL #0
I7: .FILL #0
I8: .FILL #0
I9: .FILL #0
J1: .FILL #0
J2: .FILL #0
J3: .FILL #0
J4: .FILL #0
J5: .FILL #0
J6: .FILL #0
J7: .FILL #0
J8: .FILL #0
J9: .FILL #0

;;-------------------------------------------------------------------------CP GUESS BOARD (reversed and with nulls for PUTSP)
A1ckn:    .FILL  x3141
A2ckn:    .FILL  x3241
A3ckn:    .FILL  x3341
A4ckn:    .FILL  x3441
A5ckn:    .FILL  x3541
A6ckn:    .FILL  x3641
A7ckn:    .FILL  x3741
A8ckn:    .FILL  x3841
A9ckn:    .FILL  x3941    
B1ckn:    .FILL  x3142
B2ckn:    .FILL  x3242
B3ckn:    .FILL  x3342
B4ckn:    .FILL  x3442
B5ckn:    .FILL  x3542
B6ckn:    .FILL  x3642
B7ckn:    .FILL  x3742
B8ckn:    .FILL  x3842
B9ckn:    .FILL  x3942
C1ckn:    .FILL  x3143
C2ckn: .FILL     x3243
C3ckn:  .FILL    x3343
C4ckn: .FILL     x3443
C5ckn: .FILL     x3543
C6ckn:  .FILL    x3643
C7ckn:  .FILL    x3743
C8ckn:  .FILL    x3843
C9ckn:  .FILL    x3943
D1ckn:  .FILL   x3144
D2ckn:  .FILL   x3244
D3ckn:  .FILL   x3344
D4ckn:  .FILL    x3444
D5ckn:  .FILL   x3544
D6ckn:  .FILL   x3644
D7ckn:   .FILL  x3744
D8ckn:  .FILL   x3844
D9ckn:   .FILL  x3944
E1ckn:   .FILL   x3145
E2ckn:   .FILL   x3245
E3ckn:   .FILL   x3345
E4ckn:   .FILL   x3445
E5ckn:   .FILL   x3545
E6ckn:   .FILL   x3645
E7ckn:    .FILL  x3745
E8ckn:   .FILL   x3845
E9ckn:   .FILL   x3945
F1ckn:   .FILL    x3146
F2ckn:   .FILL    x3246
F3ckn:   .FILL    x3346
F4ckn:   .FILL    x3446
F5ckn:   .FILL    x3546
F6ckn:   .FILL    x3646
F7ckn:   .FILL    x3746
F8ckn:   .FILL   x3846
F9ckn:   .FILL    x3946
G1ckn:   .FILL     x3147
G2ckn:   .FILL     x3247
G3ckn:   .FILL     x3347
G4ckn:   .FILL    x3447
G5ckn:   .FILL    x3547
G6ckn:   .FILL    x3647
G7ckn:   .FILL    x3747
G8ckn:    .FILL   x3847
G9ckn:   .FILL     x3947
H1ckn:   .FILL     x3148
H2ckn:    .FILL    x3248
H3ckn:    .FILL    x3348
H4ckn:    .FILL    x3448
H5ckn:    .FILL    x3548
H6ckn:    .FILL    x3648
H7ckn:    .FILL    x3748
H8ckn:    .FILL    x3848
H9ckn:    .FILL   x3948
I1ckn:    .FILL   x3149
I2ckn:    .FILL   x3249
I3ckn:    .FILL   x3349
I4ckn:    .FILL  x3449
I5ckn:    .FILL  x3549
I6ckn:    .FILL  x3649
I7ckn:    .FILL   x3749
I8ckn:    .FILL   x3849         
I9ckn:    .FILL     x3949
J1ckn:   .FILL     x314A
J2ckn:   .FILL     x324A
J3ckn:   .FILL     x334A
J4ckn:    .FILL    x344A
J5ckn:   .FILL     x354A
J6ckn:    .FILL    x364A
J7ckn:   .FILL     x374A
J8ckn:   .FILL     x384A
J9ckn:    .FILL    x394A   

;;------------------------------------------------------CP GUESS ALREADY (to stop repeat guesses)
POINTERa: .FILL #0
A1a: .FILL #0
A2a: .FILL #0
A3a: .FILL #0
A4a: .FILL #0
A5a: .FILL #0
A6a: .FILL #0
A7a: .FILL #0
A8a: .FILL #0
A9a: .FILL #0
B1a: .FILL #0
B2a: .FILL #0
B3a: .FILL #0
B4a: .FILL #0
B5a: .FILL #0
B6a: .FILL #0
B7a: .FILL #0
B8a: .FILL #0
B9a: .FILL #0
C1a: .FILL #0
C2a: .FILL #0
C3a: .FILL #0
C4a: .FILL #0
C5a: .FILL #0
C6a: .FILL #0
C7a: .FILL #0
C8a: .FILL #0
C9a: .FILL #0
D1a: .FILL #0
D2a: .FILL #0
D3a: .FILL #0
D4a: .FILL #0
D5a: .FILL #0
D6a: .FILL #0
D7a: .FILL #0
D8a: .FILL #0
D9a: .FILL #0
E1a: .FILL #0
E2a: .FILL #0
E3a: .FILL #0
E4a: .FILL #0
E5a: .FILL #0
E6a: .FILL #0
E7a: .FILL #0
E8a: .FILL #0
E9a: .FILL #0
F1a: .FILL #0
F2a: .FILL #0
F3a: .FILL #0
F4a: .FILL #0
F5a: .FILL #0
F6a: .FILL #0
F7a: .FILL #0
F8a: .FILL #0
F9a: .FILL #0
G1a: .FILL #0
G2a: .FILL #0
G3a: .FILL #0
G4a: .FILL #0
G5a: .FILL #0
G6a: .FILL #0
G7a: .FILL #0
G8a: .FILL #0
G9a: .FILL #0
H1a: .FILL #0
H2a: .FILL #0
H3a: .FILL #0
H4a: .FILL #0
H5a: .FILL #0
H6a: .FILL #0
H7a: .FILL #0
H8a: .FILL #0
H9a: .FILL #0
I1a: .FILL #0
I2a: .FILL #0
I3a: .FILL #0
I4a: .FILL #0
I5a: .FILL #0
I6a: .FILL #0
I7a: .FILL #0
I8a: .FILL #0
I9a: .FILL #0
J1a: .FILL #0
J2a: .FILL #0
J3a: .FILL #0
J4a: .FILL #0
J5a: .FILL #0
J6a: .FILL #0
J7a: .FILL #0
J8a: .FILL #0
J9a: .FILL #0
;;--------------------------------------------------THE SHIPS
Carrier: .FILL #5      ;;number of holes per ship
Battleship: .FILL #4
Cruiser: .FILL #3
Destroyer: .FILL #2
Submarine: .FILL #3

       
       .END

Assembled! Use %dis or %dump to examine; use %exe to run.


In [ ]:
%exe